In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

/home/aditya/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = input_data.read_data_sets("/tmp/data", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from t

In [3]:
learning_rate = 0.001
epochs = 1000
batch_size = 100

num_inputs = 784
num_classes =10
dropout = 0.75

X = tf.placeholder(tf.float32, shape = [None, num_inputs])
Y = tf.placeholder(tf.float32, shape = [None, num_classes])
keep_prob = tf.placeholder(tf.float32)

In [4]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides = [1, strides, strides, 1], padding = 'SAME')
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)

In [5]:
def maxpool2d(x, k = 2):
    return tf.nn.max_pool(x, ksize = [1,k,k,1], strides = [1, k, k, 1], padding = 'SAME')

In [6]:
def conv_net(x, weights, bias, dropout):
    x = tf.reshape(x, shape = [-1,28,28,1])
    conv1 = conv2d(x, weights['l1'], bias['l1'])
    conv1 = maxpool2d(conv1, 2)
    
    conv2 = conv2d(conv1, weights['l2'], bias['l2'])
    conv2 = maxpool2d(conv2, 2)
    
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), bias['bd1'])
    fc1 = tf.nn.relu(fc1)
    
    fc1 = tf.nn.dropout(fc1, dropout)
    out = tf.add(tf.matmul(fc1, weights['out']), bias['out'])
    return out

In [7]:
weights = {
    'l1': tf.Variable(tf.random_normal([5,5,1,32])),
    'l2': tf.Variable(tf.random_normal([5,5,32,64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out':tf.Variable(tf.random_normal([1024, num_classes]))
}
bias = {
    'l1':tf.Variable(tf.random_normal([32])),
    'l2':tf.Variable(tf.random_normal([64])),
    'bd1':tf.Variable(tf.random_normal([1024])),
    'out':tf.Variable(tf.random_normal([num_classes]))
}

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
logits = conv_net(X, weights, bias, keep_prob)

prediction = tf.nn.softmax(logits)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate = 0.01).minimize(loss)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [9]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.8})
        loss_, accuracy_ = sess.run([loss, accuracy], feed_dict={X: batch_xs,
                                                                 Y: batch_ys,
                                                                 keep_prob: 1.0})
        print("Epoch: {}..........Loss: {}...........Accuracy: {}".format(epoch+1, loss_, accuracy_))
        
    print("Overall Accuracy: {}".format(sess.run(accuracy, feed_dict={X: mnist.test.images[:256],
                                                                      Y: mnist.test.labels[:256],
                                                                      keep_prob: 1.0})))

Epoch: 1..........Loss: 93782.90625...........Accuracy: 0.17000000178813934
Epoch: 2..........Loss: 110047.2109375...........Accuracy: 0.20000000298023224
Epoch: 3..........Loss: 101565.6875...........Accuracy: 0.2199999988079071
Epoch: 4..........Loss: 80219.953125...........Accuracy: 0.23000000417232513
Epoch: 5..........Loss: 49174.421875...........Accuracy: 0.38999998569488525
Epoch: 6..........Loss: 46381.65625...........Accuracy: 0.44999998807907104
Epoch: 7..........Loss: 51674.5390625...........Accuracy: 0.46000000834465027
Epoch: 8..........Loss: 34472.328125...........Accuracy: 0.4699999988079071
Epoch: 9..........Loss: 25210.130859375...........Accuracy: 0.5199999809265137
Epoch: 10..........Loss: 25607.75...........Accuracy: 0.47999998927116394
Epoch: 11..........Loss: 14247.103515625...........Accuracy: 0.5400000214576721
Epoch: 12..........Loss: 14625.240234375...........Accuracy: 0.5600000023841858
Epoch: 13..........Loss: 10328.1484375...........Accuracy: 0.579999983310

Epoch: 103..........Loss: 167.79234313964844...........Accuracy: 0.949999988079071
Epoch: 104..........Loss: 180.76812744140625...........Accuracy: 0.9300000071525574
Epoch: 105..........Loss: 272.8663330078125...........Accuracy: 0.9300000071525574
Epoch: 106..........Loss: 217.7118682861328...........Accuracy: 0.949999988079071
Epoch: 107..........Loss: 407.9804992675781...........Accuracy: 0.9100000262260437
Epoch: 108..........Loss: 270.05584716796875...........Accuracy: 0.9399999976158142
Epoch: 109..........Loss: 414.979248046875...........Accuracy: 0.9300000071525574
Epoch: 110..........Loss: 73.87870788574219...........Accuracy: 0.9300000071525574
Epoch: 111..........Loss: 302.68975830078125...........Accuracy: 0.9200000166893005
Epoch: 112..........Loss: 435.1524963378906...........Accuracy: 0.8799999952316284
Epoch: 113..........Loss: 494.716552734375...........Accuracy: 0.8799999952316284
Epoch: 114..........Loss: 184.73411560058594...........Accuracy: 0.9599999785423279
Epo

Epoch: 202..........Loss: 163.90162658691406...........Accuracy: 0.9700000286102295
Epoch: 203..........Loss: 79.73658752441406...........Accuracy: 0.949999988079071
Epoch: 204..........Loss: 140.86135864257812...........Accuracy: 0.9599999785423279
Epoch: 205..........Loss: 219.29727172851562...........Accuracy: 0.9200000166893005
Epoch: 206..........Loss: 164.86712646484375...........Accuracy: 0.9399999976158142
Epoch: 207..........Loss: 144.3225555419922...........Accuracy: 0.949999988079071
Epoch: 208..........Loss: 64.81246948242188...........Accuracy: 0.9599999785423279
Epoch: 209..........Loss: 145.61814880371094...........Accuracy: 0.9599999785423279
Epoch: 210..........Loss: 160.6707305908203...........Accuracy: 0.9200000166893005
Epoch: 211..........Loss: 175.8008575439453...........Accuracy: 0.9300000071525574
Epoch: 212..........Loss: 53.39053726196289...........Accuracy: 0.9700000286102295
Epoch: 213..........Loss: 254.0384521484375...........Accuracy: 0.8799999952316284
E

Epoch: 301..........Loss: 64.6446762084961...........Accuracy: 0.9599999785423279
Epoch: 302..........Loss: 177.2081298828125...........Accuracy: 0.9399999976158142
Epoch: 303..........Loss: 37.822540283203125...........Accuracy: 0.9700000286102295
Epoch: 304..........Loss: 35.244510650634766...........Accuracy: 0.9900000095367432
Epoch: 305..........Loss: 144.5201416015625...........Accuracy: 0.9599999785423279
Epoch: 306..........Loss: 258.9249267578125...........Accuracy: 0.8899999856948853
Epoch: 307..........Loss: 90.70838928222656...........Accuracy: 0.9900000095367432
Epoch: 308..........Loss: 161.37686157226562...........Accuracy: 0.9399999976158142
Epoch: 309..........Loss: 100.56146240234375...........Accuracy: 0.9599999785423279
Epoch: 310..........Loss: 53.771183013916016...........Accuracy: 0.9800000190734863
Epoch: 311..........Loss: 88.99654388427734...........Accuracy: 0.9300000071525574
Epoch: 312..........Loss: 45.32219696044922...........Accuracy: 0.9800000190734863


Epoch: 401..........Loss: 25.4384708404541...........Accuracy: 0.9800000190734863
Epoch: 402..........Loss: 5.82760763168335...........Accuracy: 0.9900000095367432
Epoch: 403..........Loss: 91.67986297607422...........Accuracy: 0.9700000286102295
Epoch: 404..........Loss: 62.374267578125...........Accuracy: 0.949999988079071
Epoch: 405..........Loss: 70.44367980957031...........Accuracy: 0.9599999785423279
Epoch: 406..........Loss: 20.01024627685547...........Accuracy: 0.9800000190734863
Epoch: 407..........Loss: 55.57332992553711...........Accuracy: 0.9599999785423279
Epoch: 408..........Loss: 30.952611923217773...........Accuracy: 0.9700000286102295
Epoch: 409..........Loss: 98.04488372802734...........Accuracy: 0.949999988079071
Epoch: 410..........Loss: 191.7755126953125...........Accuracy: 0.9200000166893005
Epoch: 411..........Loss: 65.75163269042969...........Accuracy: 0.949999988079071
Epoch: 412..........Loss: 100.10822296142578...........Accuracy: 0.9599999785423279
Epoch: 41

Epoch: 501..........Loss: 34.11716842651367...........Accuracy: 0.9700000286102295
Epoch: 502..........Loss: 30.80113983154297...........Accuracy: 0.9599999785423279
Epoch: 503..........Loss: 61.435001373291016...........Accuracy: 0.9599999785423279
Epoch: 504..........Loss: 26.464370727539062...........Accuracy: 0.9700000286102295
Epoch: 505..........Loss: 92.25216674804688...........Accuracy: 0.9700000286102295
Epoch: 506..........Loss: 131.24049377441406...........Accuracy: 0.949999988079071
Epoch: 507..........Loss: 7.59014892578125...........Accuracy: 0.9700000286102295
Epoch: 508..........Loss: 27.32125473022461...........Accuracy: 0.9800000190734863
Epoch: 509..........Loss: 62.996280670166016...........Accuracy: 0.9900000095367432
Epoch: 510..........Loss: 60.779727935791016...........Accuracy: 0.9399999976158142
Epoch: 511..........Loss: 40.86491775512695...........Accuracy: 0.9599999785423279
Epoch: 512..........Loss: 151.9796142578125...........Accuracy: 0.9399999976158142
E

Epoch: 600..........Loss: 43.345977783203125...........Accuracy: 0.9599999785423279
Epoch: 601..........Loss: 0.0...........Accuracy: 1.0
Epoch: 602..........Loss: 133.10452270507812...........Accuracy: 0.9700000286102295
Epoch: 603..........Loss: 40.86354446411133...........Accuracy: 0.9599999785423279
Epoch: 604..........Loss: 25.56182098388672...........Accuracy: 0.9800000190734863
Epoch: 605..........Loss: 28.480810165405273...........Accuracy: 0.9800000190734863
Epoch: 606..........Loss: 62.23549270629883...........Accuracy: 0.9700000286102295
Epoch: 607..........Loss: 3.154355525970459...........Accuracy: 0.9900000095367432
Epoch: 608..........Loss: 150.05274963378906...........Accuracy: 0.949999988079071
Epoch: 609..........Loss: 0.0...........Accuracy: 1.0
Epoch: 610..........Loss: 30.910688400268555...........Accuracy: 0.9800000190734863
Epoch: 611..........Loss: 0.0...........Accuracy: 1.0
Epoch: 612..........Loss: 26.576322555541992...........Accuracy: 0.9800000190734863
Epo

Epoch: 702..........Loss: 13.321113586425781...........Accuracy: 0.9800000190734863
Epoch: 703..........Loss: 32.13568878173828...........Accuracy: 0.9900000095367432
Epoch: 704..........Loss: 127.95791625976562...........Accuracy: 0.949999988079071
Epoch: 705..........Loss: 40.13371276855469...........Accuracy: 0.9900000095367432
Epoch: 706..........Loss: 19.464637756347656...........Accuracy: 0.9900000095367432
Epoch: 707..........Loss: 98.21199035644531...........Accuracy: 0.9599999785423279
Epoch: 708..........Loss: 56.92123031616211...........Accuracy: 0.9700000286102295
Epoch: 709..........Loss: 45.84236145019531...........Accuracy: 0.949999988079071
Epoch: 710..........Loss: 16.173364639282227...........Accuracy: 0.9900000095367432
Epoch: 711..........Loss: 20.715303421020508...........Accuracy: 0.9800000190734863
Epoch: 712..........Loss: 158.83139038085938...........Accuracy: 0.9700000286102295
Epoch: 713..........Loss: 25.5776309967041...........Accuracy: 0.9800000190734863
E

Epoch: 802..........Loss: 77.34992980957031...........Accuracy: 0.9700000286102295
Epoch: 803..........Loss: 40.23189926147461...........Accuracy: 0.9800000190734863
Epoch: 804..........Loss: 8.770971298217773...........Accuracy: 0.9900000095367432
Epoch: 805..........Loss: 0.0...........Accuracy: 1.0
Epoch: 806..........Loss: 47.573429107666016...........Accuracy: 0.9599999785423279
Epoch: 807..........Loss: 36.93024826049805...........Accuracy: 0.949999988079071
Epoch: 808..........Loss: 25.1414852142334...........Accuracy: 0.9800000190734863
Epoch: 809..........Loss: 44.745452880859375...........Accuracy: 0.9800000190734863
Epoch: 810..........Loss: 25.016984939575195...........Accuracy: 0.9700000286102295
Epoch: 811..........Loss: 42.140472412109375...........Accuracy: 0.9700000286102295
Epoch: 812..........Loss: 58.573341369628906...........Accuracy: 0.949999988079071
Epoch: 813..........Loss: 14.98845100402832...........Accuracy: 0.9800000190734863
Epoch: 814..........Loss: 25.47

Epoch: 903..........Loss: 52.83504867553711...........Accuracy: 0.9800000190734863
Epoch: 904..........Loss: 51.086769104003906...........Accuracy: 0.9399999976158142
Epoch: 905..........Loss: 65.40782165527344...........Accuracy: 0.9800000190734863
Epoch: 906..........Loss: 15.455192565917969...........Accuracy: 0.9800000190734863
Epoch: 907..........Loss: 61.83884811401367...........Accuracy: 0.9700000286102295
Epoch: 908..........Loss: 66.4628677368164...........Accuracy: 0.9800000190734863
Epoch: 909..........Loss: 2.086604118347168...........Accuracy: 0.9900000095367432
Epoch: 910..........Loss: 16.64580535888672...........Accuracy: 0.9800000190734863
Epoch: 911..........Loss: 9.9932222366333...........Accuracy: 0.9900000095367432
Epoch: 912..........Loss: 30.106889724731445...........Accuracy: 0.9800000190734863
Epoch: 913..........Loss: 58.554073333740234...........Accuracy: 0.9700000286102295
Epoch: 914..........Loss: 16.65072250366211...........Accuracy: 0.9900000095367432
Epo

In [11]:
saver = tf.train.Saver()

In [12]:
saver.save(sess, 'model.ckpt')

RuntimeError: Attempted to use a closed Session.